![zad1.png](images/note3/zad1.png)

In [1]:
# Zadanie w pdfie (images/note3/solutions)

![zad3.png](images/note3/zad3.png)

In [7]:
import numpy as np
from scipy import stats

def read_data(filename):
    with open(filename, 'r') as f:
        # wczytujemy sobie dane jako tablicę liczb zmiennoprzecinkowych
        return np.array([float(line.strip()) for line in f])

def corrected_ks_statistic(u, N):
    # stosujemy korekty dla statystyki testowej KS zgodnie z podanym wzorem:
    # u -> u + ( 1 / (6 * sqrt(N))) + ((u-1) / (4*N)) gdzie:
    # - u: statystyka testowa KS
    # - N: liczba próbek
    return u + 1/(6*np.sqrt(N)) + (u-1)/(4*N)

# wczytujemy dane
data_a = read_data('data/dat2a.txt')
data_b = read_data('data/dat2b.txt')

# podpunkt a) sprawdzamy czy dane z dat2a.txt pochodzą z rozkładu normalnego N(5, 2)
ks_stat_a, _ = stats.kstest(data_a, 'norm', args=(5, 2))
N_a = len(data_a)
corrected_stat_a = corrected_ks_statistic(ks_stat_a, N_a)
p_value_a = stats.kstwobign.sf(corrected_stat_a * np.sqrt(N_a))

# podpunkt b) sprawdzamy czy dane z dat2b.txt pochodzą z rozkładu normalnego N(5, 2)
ks_stat_b, _ = stats.kstest(data_b, 'norm', args=(5, 2))
N_b = len(data_b)
corrected_stat_b = corrected_ks_statistic(ks_stat_b, N_b)
p_value_b = stats.kstwobign.sf(corrected_stat_b * np.sqrt(N_b))


# podkpunkt c) sprawdzamy czy dane z dat2a.txt i dat2b.txt pochodzą z tego samego rozkładu
ks_stat_ab, _ = stats.ks_2samp(data_a, data_b)
N_ab = min(len(data_a), len(data_b))
corrected_stat_ab = corrected_ks_statistic(ks_stat_ab, N_ab)
p_value_ab = stats.kstwobign.sf(corrected_stat_ab * np.sqrt(N_ab/2))


# poziom istotności 5%
alpha = 0.05

print("\nKolmogorov-Smirnov test results (α = 0.05):")

print("\na) Testing if dat2a.txt come from N(5,2):")
print(f"D_N = {ks_stat_a:.6f}")
print(f"Corrected D_N = {corrected_stat_a:.6f}")
print(f"p-value = {p_value_a:.6f}")
print(f"Conclusion: {'no' if p_value_a < alpha else 'probably yes'}")

print("\nb) Testing if dat2b.txt come from N(5,2):")
print(f"D_N = {ks_stat_b:.6f}")
print(f"Corrected D_N = {corrected_stat_b:.6f}")
print(f"p-value = {p_value_b:.6f}")
print(f"Conclusion: {'no' if p_value_b < alpha else 'probably yes'}")

print("\nc) Testing if dat2a.txt and dat2b.txt come from the same distribution:")
print(f"D_N = {ks_stat_ab:.6f}")
print(f"Corrected D_N = {corrected_stat_ab:.6f}")
print(f"p-value = {p_value_ab:.6f}")
print(f"Conclusion: {'no' if p_value_ab < alpha else 'probably yes'}")
# probably yes oznacza, że nie ma wystarczających dowodów,
# aby odrzucić hipotezę zerową (dla a i b hipoteza zerowa to hipoteza, że dane
# pochodzą z rozkładu N(5,2), a dla c hipoteza to, że dane pochodzą z tego samego
# rozkładu)
# no oznacza, że mamy wystarczające dowody, aby odrzucić hipotezę zerową
# jeśli p-value < 0.05 to odrzucamy hipoteze
# jeśli p-value ≥ 0.05 to nie odrzucamy hipotezy


Kolmogorov-Smirnov test results (α = 0.05):

a) Testing if dat2a.txt come from N(5,2):
D_N = 0.029311
Corrected D_N = 0.039883
p-value = 0.908036
Conclusion: probably yes

b) Testing if dat2b.txt come from N(5,2):
D_N = 0.105858
Corrected D_N = 0.116525
p-value = 0.008755
Conclusion: no

c) Testing if dat2a.txt and dat2b.txt come from the same distribution:
D_N = 0.115000
Corrected D_N = 0.125679
p-value = 0.084927
Conclusion: probably yes
